In [11]:
# Load the needed modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import RandomNormal, Zeros


In [32]:
# Load the data from the file
df = pd.read_csv('https://raw.githubusercontent.com/magitz/possum/refs/heads/main/possum.csv')
df.head()

,case,site,Pop,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,1,1,Vic,m,8,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
1,2,1,Vic,f,6,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0
2,3,1,Vic,f,6,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0
3,4,1,Vic,f,6,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0
4,5,1,Vic,f,2,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0


In [33]:
X=df[['chest','belly']]
y=df[['age']]


In [61]:
# Define a simple neural network model
model = Sequential([
    Dense(2, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.05), bias_initializer=Zeros()),
    Dense(1, activation='linear', kernel_initializer=RandomNormal(mean=0.0, stddev=0.05), bias_initializer=Zeros())
])

In [62]:
# Compile the model (same as before)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # Added 'mae' for accuracy
# Build the model with the input shape
# This initializes the weights of the model
model.build(input_shape=(None, X.shape[1]))  # (None, 2) for your input shape



In [63]:
# Function to calculate loss, accuracy, and gradients for given parameters
def calculate_metrics(input_data, target_data, new_weights):
    # Set model weights to the provided values
    # model.set_weights(new_weights)

    # Calculate loss and accuracy
    loss, accuracy = model.evaluate(input_data, target_data, verbose=0)

    # Calculate gradients using tf.GradientTape
    with tf.GradientTape() as tape:
        y_pred = model(input_data)  # Forward pass
        loss_value = model.compute_loss(x=input_data, y=target_data, y_pred=y_pred)  # Updated line
    gradients = tape.gradient(loss_value, model.trainable_weights)

    # Print weights and gradients in a table with clearer labels
    weights_data = []
    for layer_num, layer in enumerate(model.layers):
        layer_weights = layer.get_weights()  # Get weights for the current layer

        # Weights
        weight_values = layer_weights[0].flatten()
        num_neurons_in_layer = layer_weights[0].shape[1]
        for neuron_index in range(num_neurons_in_layer):
            for weight_index_in_neuron in range(layer_weights[0].shape[0]):
                w = weight_values[neuron_index * layer_weights[0].shape[0] + weight_index_in_neuron]
                # Access the corresponding gradient for this weight
                g = gradients[layer_num * 2].numpy().flatten()[neuron_index * layer_weights[0].shape[0] + weight_index_in_neuron]
                weights_data.append([f"Layer {layer_num + 1}, Neuron {neuron_index + 1}, Weight {weight_index_in_neuron + 1}", w, g])

        # Biases
        bias_values = layer_weights[1].flatten()
        for bias_index, b in enumerate(bias_values):
            # Access the corresponding gradient for this bias
            g = gradients[layer_num * 2 + 1].numpy().flatten()[bias_index]
            weights_data.append([f"Layer {layer_num + 1}, Neuron {bias_index + 1}, Bias", b, g])

    return loss, accuracy, gradients, weights_data

In [64]:
def evaluate_model(model, X, y):
  current_weights = model.get_weights()

  loss, accuracy, gradients, weights_data = calculate_metrics(X, y, current_weights)

  # Print the results
  print("Loss:", loss)
  print("Accuracy (MAE):", accuracy)
  df_weights = pd.DataFrame(weights_data, columns=["Parameter", "Value", "Gradient"])
  print(df_weights.to_string())

  return loss, accuracy, current_weights, gradients

loss, accuracy, current_weights, gradients = evaluate_model(model, X, y)

Loss: 18.25679588317871
Accuracy (MAE): 3.827232837677002
                     Parameter     Value  Gradient
0  Layer 1, Neuron 1, Weight 1 -0.010368  0.000000
1  Layer 1, Neuron 1, Weight 2  0.031449 -3.973519
2  Layer 1, Neuron 2, Weight 1 -0.109443  0.000000
3  Layer 1, Neuron 2, Weight 2 -0.016224 -4.800104
4      Layer 1, Neuron 1, Bias  0.000000  0.000000
5      Layer 1, Neuron 2, Bias  0.000000 -0.145044
6  Layer 2, Neuron 1, Weight 1 -0.045288  0.000000
7  Layer 2, Neuron 1, Weight 2  0.018949 -2.484919
8      Layer 2, Neuron 1, Bias  0.000000 -7.654466


In [71]:
def updated_weights(current_weights, gradients, learning_rate=0.01):

  # Update weights
  updated_weights = []
  for i, (weight, gradient) in enumerate(zip(model.trainable_weights, gradients)):
      updated_weight = weight - learning_rate * gradient
      updated_weights.append(updated_weight)

  # Set the updated weights to the model
  model.set_weights(updated_weights)

  return

learning_rate = 0.1
updated_weights(current_weights, gradients, learning_rate)

In [72]:
evaluate_model(model, X, y)

Loss: 11419.263671875
Accuracy (MAE): 106.62036895751953
                     Parameter     Value      Gradient
0  Layer 1, Neuron 1, Weight 1 -0.010368      0.000000
1  Layer 1, Neuron 1, Weight 2  1.620857   5869.250000
2  Layer 1, Neuron 2, Weight 1 -0.109443      0.000000
3  Layer 1, Neuron 2, Weight 2  1.903817   7081.191406
4      Layer 1, Neuron 1, Bias  0.000000      0.000000
5      Layer 1, Neuron 2, Bias  0.058017    215.995026
6  Layer 2, Neuron 1, Weight 1 -0.045288      0.000000
7  Layer 2, Neuron 1, Weight 2  1.012916  22713.654297
8      Layer 2, Neuron 1, Bias  3.061786    213.240723


(11419.263671875,
 106.62036895751953,
 [array([[-0.01036772,  1.6208569 ],
         [-0.10944293,  1.9038173 ]], dtype=float32),
  array([0.       , 0.0580175], dtype=float32),
  array([[-0.04528809],
         [ 1.0129164 ]], dtype=float32),
  array([3.0617864], dtype=float32)],
 [<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
  array([[   0.    , 5869.25  ],
         [   0.    , 7081.1914]], dtype=float32)>,
  <tf.Tensor: shape=(2,), dtype=float32, numpy=array([  0.     , 215.99503], dtype=float32)>,
  <tf.Tensor: shape=(2, 1), dtype=float32, numpy=
  array([[    0.   ],
         [22713.654]], dtype=float32)>,
  <tf.Tensor: shape=(1,), dtype=float32, numpy=array([213.24072], dtype=float32)>])